# Bootstrapping user preferences



Similarly to (Zhu et al., 2021), we'll evaluate a dynamic recommendation system using:

1. An oracle preference model (Generated in notebook 00-preference_model)
2. A position bias examination model
3. Bootstrapped user preferences

Step 3 is important so that we're simulating exposing users to fresh items, using their previous preferences (provided by step 1) and their examination bias (step 2) to determine a sinthethical rating matrix dataset that we'll use to compare different calibration approaches as the user gets exposed to more and more items.

We'll also simulate the timestamp of user interaction, with the following methodology:

1. We'll analyze the average delta in timestamps between consecutive ratings per user in the same dataset that the preference model was trained on (e.g: movielens) (present in data/movielens-1m/avg_str_time_dff_per_user.csv)
2. Assuming that the time_delta is normally distributed (with the mean and standart deviation being defined from the dataset), we'll randomly generate a delta_t for each user. the time_0 will be the timestamp when we start generating the user preferences.

In [1]:
import pandas as pd
from scipy.stats import expon
from tqdm import tqdm

In [2]:
import numpy as np
import math
import random

In [3]:
import sys
import os
from pathlib import Path
import pickle

sys.path.append('..')

In [4]:
from constants import ML_1M_FILLED_PATH

## Reading and setting up data

In [5]:
preference_matrix = pd.read_csv(f"../{ML_1M_FILLED_PATH}").drop(columns=["Unnamed: 0"])
preference_matrix = preference_matrix.rename(columns={"Rating": "rating"})


unique_users = list(preference_matrix["user"].unique())

unique_items = list(preference_matrix["item"].unique())


click_matrix = pd.DataFrame(columns=["user", "item", "clicked_and_examined", "clicked_at", "timestamp"])

user_to_time_delta = pd.read_csv("../data/movielens-1m/median_time_diff_per_user.csv").set_index("userId")

In [6]:


userToExpDistribution = {
    user: expon(scale=row["median_timestamp_diff"])
    for user, row in user_to_time_delta.iterrows()
}




with open("userToExpDistribution.pkl", "wb") as f:
    pickle.dump(userToExpDistribution, f)

## Functions

In [7]:
def get_user_preference_for_item(user, item, matrix):
    user_ratings = matrix[matrix["user"] == user]
    return user_ratings[user_ratings["item"] == item].rating.item()

In [8]:
def click_model(k):
    lambda_k = 1/math.log(k+1,2)
    examination_probability = random.random()
    if examination_probability <= lambda_k:
        return True
    return False

In [9]:
def get_inverse_propensity_click_score(position):
    # Given a click position, this funtion returns the invense propensity, 
    # usefull to debias the data later.
    return - 1/math.log(position+1,2)

In [10]:
userToExpDistribution[1].rvs(1)[0]

6.452334368057109

In [11]:
def get_user_feedback_for_item(user, item ,k, matrix, user_to_time_distribution, initial_time):
    # Build a mapping from user to their timestamp distribution
    preference = get_user_preference_for_item(user, item, matrix)
    observed = click_model(k)
    relevant = bool(preference)
    should_click = observed and relevant
    if (should_click):
        feedback = 1
        clicked_at = k
        # Sample one delta_t random variable following the users
        # Exponential time distribution
        timestamp = initial_time + user_to_time_distribution[user].rvs(1)[0]
    else:
        if (observed):
            # Case where an item was observed, but isn´t relevant -> negative example for BPR
            feedback = 0
        else:
            # Case where an item was neither observed or relevant -> we will ignore this training instance in this loop
            feedback = None
        clicked_at = None
        timestamp = None
        
    # If user clicked the item, record the position it was in
    # feedback = 1 if user examined and clicked, 0 if user examined and not clicked,
    # None if otherwise
    return user, item, feedback, clicked_at, timestamp

In [12]:
def map_recommendation_to_feedback(user, rec_list, matrix, user_to_up_to_date_timestamp, userToExpDistribution):
    initial_time = user_to_up_to_date_timestamp.loc[user_to_up_to_date_timestamp["user"] == user, "delta_from_start"].squeeze()
    results = []
    for idx, item in enumerate(rec_list):
        user, item, feedback, clicked_at, timestamp  = get_user_feedback_for_item(user, item, idx+1, matrix, userToExpDistribution, initial_time)
        if (timestamp is not None):
            initial_time = timestamp
        feedback = (user, item, feedback, clicked_at, timestamp)
        results.append(feedback)
    return results, initial_time

In [13]:
def random_rec(items, u, k, D):
    user_history = set(D[D["user"] == u]["item"])
    candidate_items = list(set(items) - user_history)
    return random.sample(candidate_items, k)

In [14]:
initial_date = None

In [15]:
if initial_date is None:
    initial_date = pd.Timestamp.now().timestamp()
user_to_up_to_date_timestamp = pd.DataFrame({
    "user": unique_users,
    "delta_from_start": 0.0
})
user_to_up_to_date_timestamp["timestamp_dist"] = user_to_up_to_date_timestamp["user"].map(userToExpDistribution)

In [16]:
pd.Timestamp.now().timestamp()

1757439951.581035

In [17]:
user_to_up_to_date_timestamp

,user,delta_from_start,timestamp_dist
0,1,0.0,<scipy.stats._distn_infrastructure.rv_continuo...
1,2,0.0,<scipy.stats._distn_infrastructure.rv_continuo...
2,3,0.0,<scipy.stats._distn_infrastructure.rv_continuo...
3,5,0.0,<scipy.stats._distn_infrastructure.rv_continuo...
4,6,0.0,<scipy.stats._distn_infrastructure.rv_continuo...
...,...,...,...
5284,6035,0.0,<scipy.stats._distn_infrastructure.rv_continuo...
5285,6036,0.0,<scipy.stats._distn_infrastructure.rv_continuo...
5286,6037,0.0,<scipy.stats._distn_infrastructure.rv_continuo...
5287,6039,0.0,<scipy.stats._distn_infrastructure.rv_continuo...


In [18]:
def simulate_user_feedback(user, candidate_items, click_df, preference_matrix, k, user_to_up_to_date_timestamp, userToExpDistribution):
    rec = random_rec(candidate_items, user, k, click_df)
    row, last_time = map_recommendation_to_feedback(user, rec, preference_matrix, user_to_up_to_date_timestamp, userToExpDistribution)
    user_to_up_to_date_timestamp.loc[user_to_up_to_date_timestamp["user"] == user, "delta_from_start"] = last_time
    return row, user_to_up_to_date_timestamp

In [19]:
def bootstrap_clicks(D, unique_users, unique_items, preference_matrix, userToExpDistribution, k=20, rounds=10, initial_date=None):
    """
    Given unique users and unique items, recommend up to k items to every user
    using a preference matrix as a relevancy model and using a click model
    to simulate probability of user examinating an item.

    Feedback signal will be fed to the D matrix.

    We run the boostrap process for a total of an arbitrary number of rounds,
    in order to ensure enough feedback data to train a model.
    """


    if initial_date is None:
        initial_date = pd.Timestamp.now().timestamp()
    user_to_up_to_date_timestamp = pd.DataFrame({
        "user": unique_users,
        "delta_from_start": 0.0
    })
    user_to_up_to_date_timestamp["timestamp_dist"] = user_to_up_to_date_timestamp["user"].map(userToExpDistribution)
    new_df = D.copy()#pd.DataFrame(columns=["user", "item", "clicked_and_examined", "clicked_at"])
    for round in range(rounds):
        rows_to_append = []
        for user in tqdm(unique_users, desc=f"Processing users (round {round+1}/{rounds})..."):
            row, user_to_up_to_date_timestamp = simulate_user_feedback(user, unique_items, click_matrix, preference_matrix, k, user_to_up_to_date_timestamp, userToExpDistribution)
            #row, user_to_up_to_date_timestamp = map_recommendation_to_feedback(user, recs, preference_matrix, user_to_up_to_date_timestamp)
            rows_to_append.extend(row)
        round_df = pd.DataFrame(rows_to_append, columns=new_df.columns)
        new_df = pd.concat([new_df, round_df], ignore_index=True)
    final_df = pd.concat([D, new_df])
    final_df.loc[final_df["timestamp"].notnull(), "timestamp"] += initial_date
    return final_df

In [20]:
%%time
click_matrix = bootstrap_clicks(click_matrix, unique_users, unique_items, preference_matrix, userToExpDistribution=userToExpDistribution, initial_date=0.0)

Processing users (round 1/10)...:   0%|          | 0/5289 [00:00<?, ?it/s]

Processing users (round 1/10)...: 100%|██████████| 5289/5289 [13:53<00:00,  6.34it/s]
/tmp/ipykernel_20004/3645572272.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, round_df], ignore_index=True)
Processing users (round 10/10)...: 100%|██████████| 5289/5289 [14:18<00:00,  6.16it/s]


CPU times: user 2h 22min 40s, sys: 7.71 s, total: 2h 22min 47s
Wall time: 2h 22min 38s


/tmp/ipykernel_20004/3645572272.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([D, new_df])


In [22]:
click_matrix

,user,item,clicked_and_examined,clicked_at,timestamp
0,1,100,0.0,NaN,NaN
1,1,2863,1.0,2.0,7.361022
2,1,609,0.0,NaN,NaN
3,1,2273,1.0,4.0,93.286971
4,1,2290,NaN,NaN,NaN
...,...,...,...,...,...
1057795,6040,1583,0.0,NaN,NaN
1057796,6040,2050,0.0,NaN,NaN
1057797,6040,2438,NaN,NaN,NaN
1057798,6040,832,NaN,NaN,NaN


In [23]:
click_matrix.to_csv("../data/simulation/sinthetic_data.csv", index=False)

In [24]:
click_matrix = pd.read_csv("../data/simulation/sinthetic_data.csv")

In [25]:
click_matrix[click_matrix["clicked_and_examined"].notna()]["clicked_and_examined"].value_counts(dropna=False)

clicked_and_examined
0.0    250814
1.0    120944
Name: count, dtype: int64